In [1]:
!pip install arch

     |████████████████████████████████| 808kB 7.7MB/s 


In [2]:
import os
import sys

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from arch import arch_model

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
p = print

p('Machine: {} {}\n'.format(os.uname().sysname,os.uname().machine))
p(sys.version)

# Machine: Linux x86_64

# 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
# [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


Machine: Linux x86_64

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file_="/content/drive/My Drive/SP500.xls"
file_2="/content/drive/My Drive/VIXCLS.xls"

In [6]:
from pandas import to_datetime
df=pd.DataFrame(pd.read_excel(file_))
df.observation_date=to_datetime(df.observation_date,format="%Y/%m/%d")
print(df)
df1=pd.read_excel(file_2)
df1.observation_date=to_datetime(df1.observation_date,format="%Y/%m/%d")
print(df1)

     observation_date    SP500
0          2011-04-25  1335.25
1          2011-04-26  1347.24
2          2011-04-27  1355.66
3          2011-04-28  1360.48
4          2011-04-29  1363.61
...               ...      ...
2345       2020-04-20  2823.16
2346       2020-04-21  2736.56
2347       2020-04-22  2799.31
2348       2020-04-23  2797.80
2349       2020-04-24  2836.74

[2350 rows x 2 columns]
     observation_date  VIXCLS
0          2011-04-25   15.77
1          2011-04-26   15.62
2          2011-04-27   15.35
3          2011-04-28   14.62
4          2011-04-29   14.75
...               ...     ...
2345       2020-04-20   43.83
2346       2020-04-21   45.41
2347       2020-04-22   41.98
2348       2020-04-23   41.38
2349       2020-04-24   35.93

[2350 rows x 2 columns]


In [0]:
data=pd.merge(df,df1)
data.set_index(['observation_date'], inplace=True)

In [0]:
lrets = np.log(data/data.shift(1)).dropna()

In [9]:
max_lag = 30
Y = lrets.SP500
mdl = smt.ARMA(Y, order=(1, 0)).fit(
    maxlag=max_lag, method='mle', trend='nc')
p(mdl.summary())

                              ARMA Model Results                              
Dep. Variable:                  SP500   No. Observations:                 2182
Model:                     ARMA(1, 0)   Log Likelihood                6758.660
Method:                           mle   S.D. of innovations              0.011
Date:                Thu, 30 Apr 2020   AIC                         -13513.320
Time:                        02:59:46   BIC                         -13501.944
Sample:                             0   HQIC                        -13509.162
                                                                              
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
ar.L1.SP500    -0.1659      0.021     -7.857      0.000      -0.207      -0.125
                                    Roots                                    
                  Real          Imaginary         

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
best_aic = np.inf 
best_order = None
best_mdl = None

rng = range(5) 
for i in rng:
    for j in rng:
        try:
            tmp_mdl = smt.ARMA(lrets['SP500'], order=(i, j)).fit(
                method='mle', trend='nc'
            )
            tmp_aic = tmp_mdl.aic
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_order = (i, j)
                best_mdl = tmp_mdl
        except: continue


p('aic: {:6.5f} | order: {}'.format(best_aic, best_order))

aic: -13609.08702 | order: (4, 4)


In [12]:
best_aic = np.inf 
best_order = None
best_mdl = None
pq_rng = range(5) 
d_rng = range(2) 
for i in pq_rng:
    for d in d_rng:
        for j in pq_rng:
            try:
                tmp_mdl = smt.ARIMA(lrets.SP500, order=(i,d,j)).fit(method='mle', trend='nc')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, d, j)
                    best_mdl = tmp_mdl
            except: continue
p('aic: {:6.5f} | order: {}'.format(best_aic, best_order))

aic: -13609.08702 | order: (4, 0, 4)


In [13]:
max_lag = 30
Y = lrets.SP500[:-30]
mdl = smt.ARMA(Y, order=(4, 0)).fit(
    maxlag=max_lag, method='mle', trend='nc')
p(mdl.summary())

                              ARMA Model Results                              
Dep. Variable:                  SP500   No. Observations:                 2152
Model:                     ARMA(4, 0)   Log Likelihood                6962.329
Method:                           mle   S.D. of innovations              0.010
Date:                Thu, 30 Apr 2020   AIC                         -13914.658
Time:                        03:01:28   BIC                         -13886.288
Sample:                             0   HQIC                        -13904.280
                                                                              
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
ar.L1.SP500    -0.0581      0.022     -2.682      0.007      -0.101      -0.016
ar.L2.SP500     0.0619      0.022      2.843      0.005       0.019       0.105
ar.L3.SP500    -0.0628      0.022     -2.841    

In [14]:
import numpy as np
Y_test = lrets.SP500[-30:].to_numpy()
Y_full = lrets.SP500.array
test = np.zeros((30,1))
diff_1 = np.zeros((30,1))
for i in range(30):
  test[i]=(-0.0581)*Y_full[-30+i]+0.0619*Y_full[-29+i]+(-0.0628)*Y_full[-28+i]+0.01*Y_full[-27+i]
diff_1 = np.mean(test-Y_test)
print(diff_1)
# We can trust this model

-0.0013286742935573117


In [15]:
max_lag = 30
Y = lrets.SP500
mdl = smt.ARMA(Y, order=(4, 0)).fit(
    maxlag=max_lag, method='mle', trend='nc')
p(mdl.summary())

                              ARMA Model Results                              
Dep. Variable:                  SP500   No. Observations:                 2182
Model:                     ARMA(4, 0)   Log Likelihood                6774.523
Method:                           mle   S.D. of innovations              0.011
Date:                Thu, 30 Apr 2020   AIC                         -13539.047
Time:                        03:01:28   BIC                         -13510.607
Sample:                             0   HQIC                        -13528.650
                                                                              
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
ar.L1.SP500    -0.1472      0.021     -6.883      0.000      -0.189      -0.105
ar.L2.SP500     0.1149      0.022      5.315      0.000       0.073       0.157
ar.L3.SP500    -0.0118      0.022     -0.545    

In [16]:
predict_y = np.zeros((12,1))
for i in range(4):
  predict_y[i] = Y[-i-1]
print(predict_y)

[[ 0.01382211]
 [-0.00053956]
 [ 0.0226713 ]
 [-0.03115517]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [17]:
for i in range(8):
  predict_y[i+4]=(-0.1472)*predict_y[i]+0.1149*predict_y[i+1]+(-0.0118)*predict_y[i+2]+0.01*predict_y[i+3]
print(predict_y)

[[ 0.01382211]
 [-0.00053956]
 [ 0.0226713 ]
 [-0.03115517]
 [-0.00267568]
 [ 0.00302523]
 [-0.00685512]
 [ 0.00417436]
 [ 0.00086409]
 [-0.00127358]
 [ 0.00146577]
 [-0.00048549]]


In [32]:
import math
predict_price = np.zeros((13,1))
predict_price[0]=df.iloc[-1,-1]
for i in range(12):
  predict_price[i+1]=predict_price[i]*math.exp(predict_y[i])
predict_price = np.delete(predict_price, 0, axis=0)
p(predict_price)

[[2876.22196998]
 [2874.67048223]
 [2940.58738986]
 [2850.38532269]
 [2842.76878723]
 [2851.38183968]
 [2831.90212226]
 [2843.74819675]
 [2846.20652331]
 [2842.58394881]
 [2846.75359197]
 [2845.37184342]]


In [0]:
# Since the return we got is in log form, we need to compute it into normal form


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
best_aic = np.inf 
best_order = None
best_mdl = None

rng = range(5) 
for i in rng:
    for j in rng:
        try:
            tmp_mdl = smt.ARMA(lrets['VIXCLS'], order=(i, j)).fit(
                method='mle', trend='nc'
            )
            tmp_aic = tmp_mdl.aic
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_order = (i, j)
                best_mdl = tmp_mdl
        except: continue


p('aic: {:6.5f} | order: {}'.format(best_aic, best_order))

aic: -4847.28018 | order: (2, 1)


In [20]:
best_aic = np.inf 
best_order = None
best_mdl = None
pq_rng = range(5) 
d_rng = range(2) 
for i in pq_rng:
    for d in d_rng:
        for j in pq_rng:
            try:
                tmp_mdl = smt.ARIMA(lrets.VIXCLS, order=(i,d,j)).fit(method='mle', trend='nc')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, d, j)
                    best_mdl = tmp_mdl
            except: continue
p('aic: {:6.5f} | order: {}'.format(best_aic, best_order))

aic: -4847.28018 | order: (2, 0, 1)


In [21]:
max_lag = 30
X = lrets.VIXCLS
mdl = smt.ARMA(X, order=(2, 0)).fit(
    maxlag=max_lag, method='mle', trend='nc')
p(mdl.summary())

                              ARMA Model Results                              
Dep. Variable:                 VIXCLS   No. Observations:                 2182
Model:                     ARMA(2, 0)   Log Likelihood                2416.907
Method:                           mle   S.D. of innovations              0.080
Date:                Thu, 30 Apr 2020   AIC                          -4827.814
Time:                        03:03:00   BIC                          -4810.750
Sample:                             0   HQIC                         -4821.576
                                                                              
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
ar.L1.VIXCLS    -0.0756      0.021     -3.530      0.000      -0.118      -0.034
ar.L2.VIXCLS    -0.0264      0.021     -1.233      0.218      -0.068       0.016
                                    Roots   

In [22]:
import numpy as np
X_test = lrets.VIXCLS[-30:].to_numpy()
X_full = lrets.VIXCLS.array
test = np.zeros((30,1))
diff_1 = np.zeros((30,1))
for i in range(30):
  test[i]=(-0.0756)*X_full[-30+i]+(-0.0264)*Y_full[-29+i]
diff_1 = np.mean(test-X_test)
print(diff_1)
# We can trust this model

0.013972948006683212


In [23]:
predict = np.zeros((10,1))
for i in range(2):
  predict[i] = X[-i-1]
print(predict)

[[-0.14122507]
 [-0.01439564]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [24]:
for i in range(8):
  predict[i+2]=(-0.0756)*predict[i]+(-0.0264)*predict[i+1]
print(predict)

[[-1.41225071e-01]
 [-1.43956421e-02]
 [ 1.10566603e-02]
 [ 7.96414708e-04]
 [-8.56908869e-04]
 [-3.75865578e-05]
 [ 6.57745957e-05]
 [ 1.10509444e-06]
 [-5.00173392e-06]
 [ 4.85006358e-08]]


In [38]:
predict_vix = np.zeros((11,1))
predict_vix[0]=df1.iloc[-1,-1]
for i in range(10):
  predict_vix[i+1]=predict_vix[i]*math.exp(predict[i])
predict_vix = np.delete(predict_vix, 0, axis=0)
p(predict_vix)

[[31.19779845]
 [30.75190329]
 [31.09380329]
 [31.11857672]
 [31.09192235]
 [31.09075374]
 [31.09279879]
 [31.09283315]
 [31.09267763]
 [31.09267914]]
